In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
import pyttsx3

# Define paths
capture_folder = "C:/Users/hp/Downloads/Dataset/captured_images"
model_path = "C:/Users/hp/Downloads/Dataset/pose_classification_model.h5"
good_pose_folder = "C:/Users/hp/Downloads/Dataset/images/good pose"

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Create the folder if it doesn't exist
if not os.path.exists(capture_folder):
    os.makedirs(capture_folder)

# Load pre-trained model
model = load_model(model_path)

def preprocess_img(img):
    img = cv2.resize(img, (224, 224))  # Resize image to model's input size
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def classify_pose(img):
    img = preprocess_img(img)
    prediction = model.predict(img)
    labels = ['good_pose', 'harmful_pose', 'neutral_pose']
    return labels[np.argmax(prediction)]

def compare_with_dataset(img):
    # Convert the image to a format that can be compared with dataset images
    img = cv2.resize(img, (224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    
    for filename in os.listdir(good_pose_folder):
        img_path = os.path.join(good_pose_folder, filename)
        dataset_img = cv2.imread(img_path)
        dataset_img = cv2.resize(dataset_img, (224, 224))
        dataset_img = np.array(dataset_img) / 255.0
        dataset_img = np.expand_dims(dataset_img, axis=0)
        
        # Simple comparison (could be improved with a better similarity metric)
        if np.allclose(img, dataset_img, atol=0.1):
            return True
    return False

def speak_message(message):
    engine.say(message)
    engine.runAndWait()

def capture_and_classify():
    cap = cv2.VideoCapture(0)  # Open the camera
    if not cap.isOpened():
        print("Error: Could not access the camera.")
        return

    print("Starting image capture every 3 seconds. Press 'q' to quit.")
    start_time = datetime.now()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break
        
        cv2.imshow('Captured Image', frame)
        
        if (datetime.now() - start_time).seconds >= 3:
            start_time = datetime.now()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            image_path = os.path.join(capture_folder, f"image_{timestamp}.jpg")
            cv2.imwrite(image_path, frame)
            print(f"Image saved to {image_path}")
            
            # Classify the captured image
            pose = classify_pose(frame)
            print(f"Pose classified as: {pose}")

            # Check if image matches dataset images
            if compare_with_dataset(frame):
                print("Pose matches a good pose image.")
                speak_message("Good pose detected.")
            else:
                print("Pose does not match any good pose images.")
                speak_message("Warning: Pose does not match any good pose images.")
                
            # Display the result image in a window
            cv2.imshow('Captured Pose', frame)

        key = cv2.waitKey(1)
        if key == ord('q'):  # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
capture_and_classify()


Starting image capture every 3 seconds. Press 'q' to quit.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160924.jpg


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step
Pose classified as: good_pose
Pose does not match any good pose images.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160929.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Pose classified as: good_pose
Pose does not match any good pose images.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160933.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Pose classified as: good_pose
Pose does not match any good pose images.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160937.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Pose classified as: good_pose
Pose does not match any good pose images.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160941.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Pose classified as: good_pose
Pose does not match any good pose images.
Image saved to C:/Users/hp/Downloads/Dataset/captured_images\image_20240820_160945.jpg
1/1 ━━━━━━━━━━